In [1]:
from transformers import AutoTokenizer, MixtralModel, MixtralConfig
config = MixtralConfig.from_pretrained("hf-internal-testing/Mixtral-tiny")
config.output_router_logits = True
config.output_attentions = True
model = MixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny", config=config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)

/home/adrien/Documents/flax/FlaxMixtral/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
MixtralModel is using MixtralSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


sparse block:  tensor([[ 0.0565,  0.3033,  0.0746,  0.5048, -0.1273,  0.5146,  0.3725,  0.7331],
        [-0.6925, -0.6075,  1.0916, -0.0951, -0.6929, -0.5925,  0.9077, -0.0924],
        [ 0.5982, -0.8751, -0.2029, -0.1773,  0.1937,  0.4039,  0.8775, -0.3451],
        [-0.0740, -0.7922,  0.0529,  0.9633, -0.0918,  0.3685,  0.0860, -0.0421],
        [-1.0877,  0.0867,  0.0434,  0.0448,  0.6515,  1.2938,  0.2649, -1.2108],
        [ 0.1927,  0.2432, -0.2782, -0.3695, -0.6098, -0.1551, -0.3344,  0.0463],
        [ 0.3873,  0.1157, -0.2483, -0.6784, -0.7712, -0.8781,  0.0244,  0.2367],
        [ 0.0341, -0.4348, -0.0610, -0.5961, -0.3237, -0.4529, -0.4729, -0.6828],
        [-1.0326,  0.0998, -0.0245, -0.0839,  0.7239,  1.3174,  0.2752, -1.2005],
        [-0.1753,  0.9458, -0.1117,  0.0430, -0.7539,  0.0218,  0.1420, -1.5287],
        [ 0.2883, -0.5809, -0.3837,  0.2736,  0.2022,  0.4310, -0.0177,  0.1172],
        [-0.6292,  0.3650,  0.7984, -0.7722,  0.9737, -0.1049, -0.4818, -0.3720],
 

In [ ]:
for i in x:
    print(i)

In [ ]:
x.router_logits

In [2]:
from transformers import AutoTokenizer, FlaxMixtralModel, MixtralConfig

config = MixtralConfig.from_pretrained("hf-internal-testing/Mixtral-tiny")
config.output_router_logits = True
# config.output_attentions = True
# config.past_key_value = True
model = FlaxMixtralModel.from_pretrained("hf-internal-testing/Mixtral-tiny", config=config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

y = model(**inputs)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


layer_outputs:  (Array([[[-0.30760983,  0.00448915, -1.7763262 , ...,  1.4183437 ,
          0.651004  , -0.4189158 ]]], dtype=float32),)
layer_outputs:  (Array([[[-1.094088 ,  1.0546091, -3.0550554, ...,  2.038435 ,
         -0.6232225, -1.5848573]]], dtype=float32),)


Some weights of the model checkpoint at hf-internal-testing/Mixtral-tiny were not used when initializing FlaxMixtralModel: {('lm_head', 'kernel')}
- This IS expected if you are initializing FlaxMixtralModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxMixtralModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some of the weights of FlaxMixtralModel were initialized in float16 precision from the model checkpoint at hf-internal-testing/Mixtral-tiny:
[('embed_tokens', 'embedding'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w1', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w2', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w3', 'kernel'), ('laye

layer_outputs:  (Array([[[-0.9512765 ,  1.3742137 ,  2.0909903 , ...,  1.8151293 ,
         -2.2085001 ,  1.4656765 ],
        [-1.0045583 , -1.0591412 , -0.16560225, ...,  0.54093945,
          0.43313074,  0.41004956],
        [-1.5399573 ,  0.14566039, -0.18943106, ..., -0.7365563 ,
         -0.6916512 , -0.4430975 ],
        ...,
        [ 0.44989714, -1.0703536 ,  0.5446242 , ..., -0.7091343 ,
         -1.0767226 ,  0.69930696],
        [-0.14029716, -0.5699066 ,  1.3213382 , ..., -1.2383035 ,
          0.9742793 , -0.3317961 ],
        [ 0.6119936 ,  0.39994478,  0.2614372 , ...,  1.2715026 ,
          0.08781135, -0.5480071 ]]], dtype=float32),)
layer_outputs:  (Array([[[-1.6238902 ,  1.354863  ,  2.5570717 , ...,  1.5965238 ,
         -2.1777525 ,  2.5399387 ],
        [-1.4560984 , -1.3028417 , -0.1199453 , ...,  0.46587005,
          0.39235753,  0.88974196],
        [-2.1907694 ,  0.08452363, -0.20379332, ..., -0.9076951 ,
         -0.70006883, -0.08643994],
        ...,
   

In [ ]:
y.router_logits

In [ ]:
y.last_hidden_state

In [ ]:
for i in y:
    print(i)

In [ ]:
from transformers import AutoTokenizer, FlaxMixtralForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")
model = FlaxMixtralForCausalLM.from_pretrained("hf-internal-testing/Mixtral-tiny")

inputs = tokenizer("Hello, my dog is cute", return_tensors="jax")
outputs = model(**inputs)

# retrieve logts for next token
# next_token_logits = outputs.logits[:, -1]
# next_token_logits

In [ ]:
for i in outputs:
    print(i)